# MVP 2: Contains the ODFM and the OC_DFGs

In [1]:
# Loading the required libraries
from pybeamline.algorithms.discovery.oc_heuristics_miner_lossy_counting import oc_heuristics_miner_lossy_counting
from pybeamline.algorithms.oc_operator import OCOperator
from pybeamline.algorithms.ocdfg_merge_operator import ocdfg_merge_operator
from pybeamline.objects.ocdfgvisualizer import OCDFGVisualizer
from pybeamline.sources.dict_ocel_test_source import dict_test_ocel_source
from pybeamline.utils.ocdfg_merger import OCDFGMerger

In [2]:
test_events_phaseflow = [
    {"activity": "Register Customer", "objects": {"Customer": ["c1"]}},
    {"activity": "Create Order", "objects": {"Customer": ["c1"], "Order": ["o1"]}},
    {"activity": "Add Item", "objects": {"Order": ["o1"], "Item": ["i1"]}},
    {"activity": "Reserve Item", "objects": {"Item": ["i1"]}},
    {"activity": "Pack Item", "objects": {"Item": ["i1"], "Order": ["o1"]}},
    {"activity": "Ship Item", "objects": {"Item": ["i1"], "Shipment": ["s1"]}},
    {"activity": "Send Invoice", "objects": {"Order": ["o1"], "Invoice": ["inv1"]}},
    {"activity": "Receive Review", "objects": {"Customer": ["c1"], "Order": ["o1"]}},
]

test_events_phaseflow_ends_early = [
    {"activity": "Register Customer", "objects": {"Customer": ["c2"]}},
    {"activity": "Create Order", "objects": {"Customer": ["c2"], "Order": ["o2"]}},
    {"activity": "Add Item", "objects": {"Order": ["o2"], "Item": ["i2"]}},
    {"activity": "Reserve Item", "objects": {"Item": ["i2"]}},
    {"activity": "Cancel Order", "objects": {"Customer": ["c2"], "Order": ["o2"]}}
]

combined_log = dict_test_ocel_source([(test_events_phaseflow_ends_early,25),(test_events_phaseflow, 2500)], shuffle=False)



In [3]:
control_flow = {
    "Order": oc_heuristics_miner_lossy_counting(model_update_frequency=10, max_approx_error=0.1),
    "Item": oc_heuristics_miner_lossy_counting(model_update_frequency=10),
    "Customer": oc_heuristics_miner_lossy_counting(model_update_frequency=10, max_approx_error=0.1),
    "Shipment": oc_heuristics_miner_lossy_counting(model_update_frequency=1),
    "Invoice": oc_heuristics_miner_lossy_counting(model_update_frequency=1),
}

In [4]:

# Object-Centric Operators
oc_operator = OCOperator(control_flow)
oc_merger = OCDFGMerger()
oc_visualizer = OCDFGVisualizer()

# Save states of changes
oc_merger.on_change(lambda odfm: oc_visualizer.save(
    odfm,
    "ocdfg_structure.png",
))

# pipe the combined log to the OCOperator op
combined_log.pipe(
    oc_operator.op(),
    ocdfg_merge_operator(oc_merger)
).subscribe()

# Visualize the ODFM in GIF
oc_visualizer.export_gif("ocdfg_evolution.gif", duration=1500)


[ODFM — Definition 8] - Object-Centric Process Mining: Dealing with Divergence and Convergence...
Create Order --(Customer)--> Cancel Order
Register Customer --(Customer)--> Create Order

[ODFM — Definition 8] - Object-Centric Process Mining: Dealing with Divergence and Convergence...
Add Item --(Order)--> Cancel Order
Create Order --(Customer)--> Cancel Order
Create Order --(Order)--> Add Item
Register Customer --(Customer)--> Create Order

[ODFM — Definition 8] - Object-Centric Process Mining: Dealing with Divergence and Convergence...
Add Item --(Item)--> Reserve Item
Create Order --(Customer)--> Cancel Order
Register Customer --(Customer)--> Create Order
Add Item --(Order)--> Cancel Order
Create Order --(Order)--> Add Item

[ODFM — Definition 8] - Object-Centric Process Mining: Dealing with Divergence and Convergence...
Add Item --(Order)--> Pack Item
Pack Item --(Order)--> Send Invoice
Add Item --(Item)--> Reserve Item
Create Order --(Customer)--> Cancel Order
Register Customer -

![OCDFG Evolution](ocdfg_evolution.gif)